In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [60]:
# Store csv created in part one into a DataFrame
df = pd.read_csv('../outputs/CityWeather.csv')
df = df.drop(columns='Unnamed: 0')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
coordinates = []
for place in df.index:
    lat, lon = df.loc[place,['lat','lon']]
    coord = (lat,lon)
    coordinates.append(coord)    

In [52]:
coordinates

[(40.5982, -124.1573),
 (-23.1203, -134.9692),
 (42.6001, -73.9662),
 (-46.1927, 168.8643),
 (5.4112, 100.3354),
 (-17.9333, 19.7667),
 (77.484, -69.3632),
 (-53.15, -70.9167),
 (-21.2, -175.2),
 (-20.4081, 57.7),
 (-34.4187, 19.2345),
 (-21.4833, 27.4833),
 (50.6996, -127.4199),
 (-9.8, -139.0333),
 (-46.6, 168.3333),
 (22.5667, 59.5289),
 (22.8909, -109.9124),
 (10.9995, 122.6711),
 (73.5069, 80.5464),
 (-46.1333, 167.6833),
 (22.0752, -159.319),
 (57.79, -152.4072),
 (35.7333, 140.8333),
 (-32.5, 137.7667),
 (9.51, -7.5692),
 (60.0625, 71.4789),
 (43.3108, 3.4758),
 (38.421, -121.4238),
 (-20.3428, 57.7575),
 (-21.6403, -41.0511),
 (43.1905, -112.345),
 (-54.8, -68.3),
 (64.3833, -173.3),
 (32.6881, 128.8419),
 (-24.7911, -50.0119),
 (-30.1811, -50.2056),
 (-22.9661, -42.0278),
 (42.097, -79.2353),
 (-33.918, 25.5701),
 (-10.4544, 161.9205),
 (64.8945, -23.7142),
 (37.8486, 40.6658),
 (-22.4184, 27.4013),
 (47.6666, -52.7314),
 (48.0056, 91.6419),
 (-32.7167, 152.15),
 (-28.7667, 11

In [58]:
# Configure gmaps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, map_type='HYBRID')
heatlayer = gmaps.heatmap_layer(coordinates,weights=df['humid'].values)
fig.add_layer(heatlayer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [98]:
# Narrow down cities that fit criteria and drop any results with null values
df_summary = df.describe(percentiles=[.1,.25,.3,.5,.75,.9])
df_summary

,maxtemp,temp,humid,cloud,wind,date,lon,lat
count,547.000000,547.000000,547.000000,547.000000,547.000000,5.470000e+02,547.000000,547.000000
mean,15.988995,15.750457,67.725777,56.080439,3.607495,1.618688e+09,15.798183,19.542107
std,12.384205,12.386711,22.448511,39.469803,2.428343,7.783197e+01,90.034133,32.882084
min,-30.000000,-30.000000,1.000000,0.000000,0.360000,1.618687e+09,-175.200000,-54.800000
10%,-1.106000,-1.136000,34.600000,0.000000,1.030000,1.618688e+09,-99.803980,-28.501760
25%,9.000000,8.370000,54.000000,18.500000,1.970000,1.618688e+09,-61.955350,-8.016000
30%,11.000000,10.540000,58.000000,20.000000,2.060000,1.618688e+09,-52.015720,-2.038920
50%,18.330000,18.000000,72.000000,71.000000,3.090000,1.618688e+09,20.252100,22.146100
75%,26.000000,25.605000,86.000000,99.000000,4.695000,1.618688e+09,97.322950,44.270600
90%,28.692000,28.542000,94.000000,100.000000,6.770000,1.618688e+09,140.313320,63.565860


In [115]:
bound_temp_lo = df_summary.loc['75%','temp']
bound_temp_hi = df_summary.loc['90%','temp']
bound_cloud = df_summary.loc['30%','cloud']
bound_wind = df_summary.loc['25%','wind']
print(bound_cloud, bound_temp,bound_wind)

20.0 28.542 1.97


In [116]:
df_vacay = df.loc[(df['temp'] >= bound_temp_lo) & (df['temp'] <= bound_temp_hi) & (df['cloud'] <= bound_cloud) & (df['wind'] <= bound_wind),]

In [122]:
df_vacay

,maxtemp,Hotel Name,temp,humid,cloud,wind,country,date,lon,lat,cityname
4,27.78,Empty,26.85,89,20,1.54,MY,1618687671,100.3354,5.4112,georgetown
127,27.00,Empty,27.00,74,17,0.51,CK,1618687570,-159.7750,-21.2078,avarua
366,26.56,Empty,26.56,80,0,1.74,PK,1618687690,62.3254,25.1216,gwadar
371,26.00,Empty,26.00,94,20,0.51,ID,1618687770,125.1824,1.4451,bitung
448,27.12,Empty,27.12,73,0,0.87,SO,1618687790,49.1816,11.2842,bosaso


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [118]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = pd.DataFrame(df_vacay)

In [119]:
hotel_df.insert(1,"Hotel Name",'Empty')

In [188]:
hotel_df = hotel_df.reset_index()

In [151]:
# Set parameters to search for a hotel
# geocoordinates
hotel_coordinates = []
target_radius = 8047
target_type = "lodging"
responsieola = []


# Iterate through 

    # get lat, lng from df
for place in hotel_df.index:
    lat, lon = hotel_df.loc[place,['lat','lon']]
    coord = f'{lat},{lon}'
    hotel_coordinates.append(coord)    
    params = {
    "location": coord,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    # Use the search term: "Hotel" and our lat/lng

    # make request and print url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(base_url, params=params)
    places_data = response.json()
    responsieola.append(places_data)
    print(json.dumps(places_data, indent=4, sort_keys=True))
    # convert to json
    
    # Grab the first hotel from the results and store the name




{
    "html_attributions": [],
    "next_page_token": "ATtYBwJzeW_2rhWFec5OJQ64brjEWWV1bRW-596VUafEhXptb-cLHK6etz0zpC0oGnCN1EDEyyiODtn0pb7hoZpyC4ofIXwPEGncLcdSGeoQDMbrSDzI5bVjewJiuE9LEYHcqsKU31-G8wdvLJtdL5EyP5uW9LgOeIV6xxAQ8CsYw5-z_9pG862TMZ8kadOqbORjHEgRQxuHazHVxZ8yzTq4LQfavQ2rjghAhBh4_4uIyAEuUz6rXsWPyjfAGL7wUWskJxx8lyXeJiAE_92Z4VF-XRSi-pGZnvOAPTaM31-B3aqShHGJ7brAHhWnGWcLP-kPgXfJDFhG-jkZpPzke3sX6vQis92ZbmlRK0F4vmwxJ7CeHk-1khz2nCV7rPS5qFzpqgjqtSpr1sJEcggRqQmIF0U3N_JALJM4GObWoTTEbVYXGcvAwCaiVyj3",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 5.4214039,
                    "lng": 100.332934
                },
                "viewport": {
                    "northeast": {
                        "lat": 5.422574580291502,
                        "lng": 100.3344920802915
                    },
                    "southwest": {
                        "lat": 5.419876619708497,
  

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIyLSOhorJDV4Y2gLdVB62eDMrWPsxayLbKvkLExr6ITOwSSxiqTRcsCfUPqa6Kk1hqJbO3ioKH73wJbnInq6VmrQYF_cum9OfTcs6NO9PzzmskEscgEIh1Rcnkuiy3RrQhvUgqFKzd9p5CYRVC6NgMkNR8uUdvuofTEuJAYs4PANLkIwqZ4_FcvG043wLk8FDmSd-KgKgTftmrKfVR6Vr1y1jqP9y_yajuSQ2robcC40ROXzCfontjj4Ol_G7UVfPhyQGH_37NMPkp3g4K-Wn-oZ9NMyHPORRGvpmWIuCYX1_0zzjLusFbwPlPBn8O_xP732NFp5n_H5MKwwYc9Y66VB3e3dov67P17St37VmYXu51UbOOXR2MsPcKyuJTRusFpjYv4HyRpFPUJQK_V_dAaHklADnD0Zx_S_w-8yjh17ap2dxCtdW4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.2128369,
                    "lng": -159.8267852
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.21159786970849,
                        "lng": -159.8255293697085
                    },
                    "southwest": {
                        "lat": -21.21429583029

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI8qCCr7qZNZoCVCBfyfooNJclooOulpqWPwpNtmaEsYMXEHudltWCH4I9FOEOOpfGDi3FZB1Htr2GtKBLViswSDp4edFi-3UF_gVCXA4A60veF_WEw7eQxYrgLdd-t2UPYBTgFateoDWBJ9uJdDJz59XCN5Qc_NPDv9v2lNgGEzRRWK7qFdMGfiNkrlPywXM3e1cZa8NT-iLkhLMTL9O-DgI1OVS7LHQD-Dq4pkyvBXp4J_KwZpnlg8fMJCU-yuo9rPHAftrhWg1yedlRUQZ-t7ZQasmBy0wt_8zlbP-hiNfl2zqWG7vpQXN0fqOhrCNz8yx1UlXg7kuITMwCdTEUp_bbT7bSs6a7pWHm1U28UENqIjgVPBybQP1NpAjzFxqc6MM92Opb34JgRwn-4JQevL9YjHrC0CfEi8R0j_mMIkjhy2iQ3Uc4V",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.125837,
                    "lng": 62.3195751
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.1271592802915,
                        "lng": 62.3209230802915
                    },
                    "southwest": {
                        "lat": 25.1244613197085,
     

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIDNqmMRiZBEdf-GIHg-kvDZlCiKFfCSu24Htb1ftu4ne7EZhJ2rRQ1RWvhwnc1fYHeL3spf6FbNchoqzK1TMmlN6uJkeWjIYNdoJOLcfLucbS5U_XgaJfKB9V5EF0NjkOjbKmIWjMypAxGK_CwjqkpX-T8DbdLADd7yma7vxAESZj4JJTrnq5ed3QkHTxHa9FxIoZylju5hGI2vOQgmtluH5-P0ILyPcvgrDIKkcONImtz_e-bvtaEtXaPUbWF_41jrcYDH6UXEIQQCRKoCJl46g8-ddOvGGS_MCEHvhe1K5C0MomIrhIALd5SOEqW4t4bQcV4ZMk9JyELMvpmfLkarC-sWIEobsnw-9PP1R9NJ00-DhP06GYAK2e55ZYfQBcO-eHoxha3p0A0GqQMKf6OiHe-FacxQr8PoGRD_aO1nMcuoQMgUdPF",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 1.4389021,
                    "lng": 125.2252508
                },
                "viewport": {
                    "northeast": {
                        "lat": 1.439729330291502,
                        "lng": 125.2269410302915
                    },
                    "southwest": {
                        "lat": 1.437031369708498,
 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKzmz-gd7SG2KMEVseaZ5xsahFTMwiSjV4Qnmxk49A71lOb_QTLFluwH8Ktk5_iwY3RzxOb7typowHNuHcLE5_xSkZJupqVf42YUXxcBAvL4O5zgKwWczobVacjYmVgVQQXMOucTk2UGUg4KrInWlSILEMVKniIZgcvFOfLySaA-TWmYDQXY6Jwl3Y2vE3jCnw47NUuO18gYUgFN6VGmqPchly8g8OHeSZHFuCmsyHRKYuUzqC5khei-_oi37H92__Bba4Yk9zH_edwRq_QiBfpcgRTjXT4XfmQ-9fglq--Bp5fFaUP5GAUFq7oCv247Zn50_Nok6Bb7r31UaUTY4J7A1icTtz-BrMHbCje_fWZWSA9SeGBgwUjtOUApA1HuV2EYcjmTILjkwaEU1vlFyrE3LDiousHtUC2kNkdnn5DOOIBtwU5XVvJ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 11.2853253,
                    "lng": 49.17900069999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 11.2866437302915,
                        "lng": 49.18037983029149
                    },
                    "southwest": {
                        "lat": 11.28394576970

In [201]:
for i in hotel_df.index:
    hotel_df['Hotel Name'][i] = responsieola[i]['results'][0]['name']

<ipython-input-201-9652ddbf5f55>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'][i] = responsieola[i]['results'][0]['name']


In [203]:
 hotel_df

,index,maxtemp,Hotel Name,temp,humid,cloud,wind,country,date,lon,lat,cityname
0,4,27.78,Cititel Penang,26.85,89,20,1.54,MY,1618687671,100.3354,5.4112,georgetown
1,127,27.00,Sunset Resort,27.00,74,17,0.51,CK,1618687570,-159.7750,-21.2078,avarua
2,366,26.56,Sadaf Resort,26.56,80,0,1.74,PK,1618687690,62.3254,25.1216,gwadar
3,371,26.00,K2 Lembeh,26.00,94,20,0.51,ID,1618687770,125.1824,1.4451,bitung
4,448,27.12,Red Sea Hotel,27.12,73,0,0.87,SO,1618687790,49.1816,11.2842,bosaso


In [206]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{cityname}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [211]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…